# Loading up the Accounting Deferred Workbook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import pickle

### Loading the historical deferred balances by BU and Major Product Config

##### NOTE: The historical data includes Marketo and Magento (whereas these are not included on the other sheets

Need to make the following changes
 - 'External Reporting BU' needs to be billed forward
 - Totals need to be eliminated
 - delete unnecessary rows
 
 ###### Note: There are excess rows in the dataframe for digital media subscriptions that are hidden. Need to remove
  - There are leading zeros in the BUs and MPCs
  

# FOR TODAY 6/8/20 Skip the whole deferred trend. Just get to the forecast

### Loading the Deferred Revenue Forecast Sheet

#### Note: For today, simply take the totals by each BU by quarter and move on!

To be done
 - clear out rows below the Grand Total inclusive of Magento/Marketo
 - forward fill the External Reporting BU
 - Move Marketo and Magento BU to Digital Experience
 - Aggregate this by External reporting BU
 - rename columns without that '
 - create interpolated periods here for the amortization (assume amortization to revenue is linear within the periods of a quarter
 - Save and move on
 
##### Note: There are a lot of rows at the bottom that need to be cleaned up

In [2]:
df = pd.read_excel("../data/Data_2020_P09/Q3'20 Rev Acctg Mgmt Workbook (09-03-20).xlsx", 
                   sheet_name='Deferred Revenue Forecast', skiprows=5)

In [3]:
df.head(10)

,External Reporting BU,Major Product Config,Historical,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22+,As Performed / Upon Acceptance,Total,Unnamed: 14
0,Digital Media,Consulting,183.899730,1.527724e+03,224.05618,110.09881,59.27300,16.32746,25.43885,0.00000,0.00000,0.000000,0.000000,1.962918e+03,NaN
1,NaN,Educational,0.000000,0.000000e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000e+00,NaN
2,NaN,Full,0.829436,4.481436e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,36923.312234,829.435830,3.775723e+04,NaN
3,NaN,Licensing,0.896455,8.964549e-01,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,895.558415,8.964549e+02,NaN
4,NaN,Licensing M&S,0.000000,5.996449e+02,370.29315,310.06177,16.17259,2.62307,0.00000,0.00000,0.00000,0.000000,0.000000,1.298795e+03,NaN
5,NaN,Other,0.000000,0.000000e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000e+00,NaN
6,NaN,Royalty,0.000000,3.338100e+02,554.48346,146.79264,73.73343,34.68510,3.11430,0.27379,0.00000,0.000000,0.000000,1.146893e+03,NaN
7,NaN,Royalty M&S,0.000000,9.645916e+02,847.60214,610.15103,94.20730,44.93262,0.02307,0.01955,0.01034,0.012210,0.000000,2.561550e+03,NaN
8,NaN,Software as Service,0.000000,4.898567e+04,35893.39215,21404.91774,8546.04991,581.84071,538.04515,560.86479,540.80982,1684.782600,0.000000,1.187364e+05,NaN
9,NaN,Subscription,5.100702,1.024024e+06,663386.02118,380069.01701,137600.67999,10589.84500,6499.65730,4863.00824,3888.50435,28024.106208,5100.701980,2.264045e+06,NaN


In [4]:
df['External Reporting BU'].value_counts(dropna=False)

NaN                                         31
Magento Deferred                             1
 Digital Media Total                         1
Marketo Deferred                             1
Digital Media                                1
Digital Experience                           1
Publishing                                   1
 Digital Experience Total                    1
 Grand Total                                 1
 Publishing Total                            1
Grand Total inclusive of Magento/Marketo     1
Name: External Reporting BU, dtype: int64

In [5]:
df.columns

Index(['External Reporting BU', 'Major Product Config', ' Historical', 'Q4'20',
       'Q1'21', 'Q2'21', 'Q3'21', 'Q4'21', 'Q1'22', 'Q2'22', 'Q3'22', 'Q4'22+',
       'As Performed / Upon Acceptance', 'Total', 'Unnamed: 14'],
      dtype='object')

In [6]:
df['External Reporting BU'].value_counts(dropna=False)

NaN                                         31
Magento Deferred                             1
 Digital Media Total                         1
Marketo Deferred                             1
Digital Media                                1
Digital Experience                           1
Publishing                                   1
 Digital Experience Total                    1
 Grand Total                                 1
 Publishing Total                            1
Grand Total inclusive of Magento/Marketo     1
Name: External Reporting BU, dtype: int64

##### Stripping spaces in the External Reporting BU columns

In [7]:
df['External Reporting BU'] = df['External Reporting BU'].str.strip()

In [8]:
df.head(50)

,External Reporting BU,Major Product Config,Historical,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22+,As Performed / Upon Acceptance,Total,Unnamed: 14
0,Digital Media,Consulting,183.899730,1.527724e+03,224.05618,110.09881,59.27300,16.32746,25.43885,0.00000,0.00000,0.000000,0.000000,1.962918e+03,NaN
1,NaN,Educational,0.000000,0.000000e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000e+00,NaN
2,NaN,Full,0.829436,4.481436e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,36923.312234,829.435830,3.775723e+04,NaN
3,NaN,Licensing,0.896455,8.964549e-01,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,895.558415,8.964549e+02,NaN
4,NaN,Licensing M&S,0.000000,5.996449e+02,370.29315,310.06177,16.17259,2.62307,0.00000,0.00000,0.00000,0.000000,0.000000,1.298795e+03,NaN
5,NaN,Other,0.000000,0.000000e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000e+00,NaN
6,NaN,Royalty,0.000000,3.338100e+02,554.48346,146.79264,73.73343,34.68510,3.11430,0.27379,0.00000,0.000000,0.000000,1.146893e+03,NaN
7,NaN,Royalty M&S,0.000000,9.645916e+02,847.60214,610.15103,94.20730,44.93262,0.02307,0.01955,0.01034,0.012210,0.000000,2.561550e+03,NaN
8,NaN,Software as Service,0.000000,4.898567e+04,35893.39215,21404.91774,8546.04991,581.84071,538.04515,560.86479,540.80982,1684.782600,0.000000,1.187364e+05,NaN
9,NaN,Subscription,5.100702,1.024024e+06,663386.02118,380069.01701,137600.67999,10589.84500,6499.65730,4863.00824,3888.50435,28024.106208,5100.701980,2.264045e+06,NaN


##### Clearing out the rows below that we do not need

In [9]:
end_loc = df[df['External Reporting BU']=='Grand Total inclusive of Magento/Marketo']
end_index = end_loc.index[0]

df = df[df.index <= end_index]

In [10]:
df['External Reporting BU'].value_counts()

Magento Deferred                            1
Publishing                                  1
Publishing Total                            1
Digital Media Total                         1
Digital Experience Total                    1
Digital Media                               1
Digital Experience                          1
Grand Total inclusive of Magento/Marketo    1
Grand Total                                 1
Marketo Deferred                            1
Name: External Reporting BU, dtype: int64

### We are just taking the following rows
- Digital Media Total
- Publishing Total
- Digital Experience Total
- Marketo Deferred
- Magento Deferred

Then we need to add the Marketo and Magento defered to the digital experience total


In [11]:
keeper_rows = ['Digital Media Total',
              'Publishing Total',
              'Digital Experience Total',
              'Marketo Deferred',
              'Magento Deferred',
              'Grand Total inclusive of Magento/Marketo']

df = df[df['External Reporting BU'].isin(keeper_rows)]
df.head(10)

,External Reporting BU,Major Product Config,Historical,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22+,As Performed / Upon Acceptance,Total,Unnamed: 14
13,Digital Media Total,NaN,190.726323,1.077714e+06,702338.98951,403290.97126,146782.43490,11452.84639,7186.77961,5490.00367,4452.80229,66650.186572,6825.696225,2.432185e+06,NaN
23,Digital Experience Total,NaN,7013.354944,3.593437e+05,203181.07510,92101.90304,32102.32943,8989.15503,12446.79748,2617.20363,2372.66414,4551.250486,2979.434210,7.206855e+05,NaN
36,Publishing Total,NaN,150.333330,2.894870e+04,19649.15565,11566.49067,6730.47778,961.91266,295.11758,140.10942,69.10113,18.677990,0.000000,6.837974e+04,NaN
38,Magento Deferred,NaN,19.491160,3.497726e+04,21749.96103,12762.51346,4655.10681,1250.99980,660.27517,745.04432,576.33967,655.035250,79.645130,7.811218e+04,NaN
39,Marketo Deferred,NaN,NaN,4.999213e+04,40848.08746,31414.17946,23461.84333,40.06227,418.52159,36.48702,19.67807,38.461460,2597.078730,1.488665e+05,NaN
40,Grand Total inclusive of Magento/Marketo,NaN,7373.905757,1.550976e+06,987767.26875,551136.05789,213732.19225,22694.97615,21007.49143,9028.84806,7490.58530,71913.611758,12481.854295,3.448228e+06,0.00181


# Cleaning out bad columns

In [12]:
df.columns

Index(['External Reporting BU', 'Major Product Config', ' Historical', 'Q4'20',
       'Q1'21', 'Q2'21', 'Q3'21', 'Q4'21', 'Q1'22', 'Q2'22', 'Q3'22', 'Q4'22+',
       'As Performed / Upon Acceptance', 'Total', 'Unnamed: 14'],
      dtype='object')

In [13]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns= ['Major Product Config', ' Historical'])

In [14]:
df.head(10)

,External Reporting BU,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22+,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.077714e+06,702338.98951,403290.97126,146782.43490,11452.84639,7186.77961,5490.00367,4452.80229,66650.186572,6825.696225,2.432185e+06
23,Digital Experience Total,3.593437e+05,203181.07510,92101.90304,32102.32943,8989.15503,12446.79748,2617.20363,2372.66414,4551.250486,2979.434210,7.206855e+05
36,Publishing Total,2.894870e+04,19649.15565,11566.49067,6730.47778,961.91266,295.11758,140.10942,69.10113,18.677990,0.000000,6.837974e+04
38,Magento Deferred,3.497726e+04,21749.96103,12762.51346,4655.10681,1250.99980,660.27517,745.04432,576.33967,655.035250,79.645130,7.811218e+04
39,Marketo Deferred,4.999213e+04,40848.08746,31414.17946,23461.84333,40.06227,418.52159,36.48702,19.67807,38.461460,2597.078730,1.488665e+05
40,Grand Total inclusive of Magento/Marketo,1.550976e+06,987767.26875,551136.05789,213732.19225,22694.97615,21007.49143,9028.84806,7490.58530,71913.611758,12481.854295,3.448228e+06


## Add Magento and Marketo to Digital Experience

##### NOTE: The External Reporting BU is different the the BU we have in deferred.
We will have to combine Creative and Document Cloud to get to the External Reporting BU since both show up as Digital Media in this accounting workbook


In [15]:
config_dict = {"str_replace_from": ["Magento Deferred", "Marketo Deferred"],
        "str_replace_to": "Digital Experience Total"}
config_dict
        

{'str_replace_from': ['Magento Deferred', 'Marketo Deferred'],
 'str_replace_to': 'Digital Experience Total'}

In [16]:
list_BU_changes = config_dict["str_replace_from"]
list_BU_new = config_dict["str_replace_to"]
    
for item in list_BU_changes:
    df["External Reporting BU"] = df["External Reporting BU"].str.replace(item, list_BU_new)


df_test['External Reporting BU'] = df_test['External Reporting BU'].str.replace('Marketo Deferred', 
                                                                               'Digital Experience Total')

df_test['External Reporting BU'] = df_test['External Reporting BU'].str.replace('Magento Deferred', 
                                                                               'Digital Experience Total')

In [17]:
df

,External Reporting BU,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22+,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.077714e+06,702338.98951,403290.97126,146782.43490,11452.84639,7186.77961,5490.00367,4452.80229,66650.186572,6825.696225,2.432185e+06
23,Digital Experience Total,3.593437e+05,203181.07510,92101.90304,32102.32943,8989.15503,12446.79748,2617.20363,2372.66414,4551.250486,2979.434210,7.206855e+05
36,Publishing Total,2.894870e+04,19649.15565,11566.49067,6730.47778,961.91266,295.11758,140.10942,69.10113,18.677990,0.000000,6.837974e+04
38,Digital Experience Total,3.497726e+04,21749.96103,12762.51346,4655.10681,1250.99980,660.27517,745.04432,576.33967,655.035250,79.645130,7.811218e+04
39,Digital Experience Total,4.999213e+04,40848.08746,31414.17946,23461.84333,40.06227,418.52159,36.48702,19.67807,38.461460,2597.078730,1.488665e+05
40,Grand Total inclusive of Magento/Marketo,1.550976e+06,987767.26875,551136.05789,213732.19225,22694.97615,21007.49143,9028.84806,7490.58530,71913.611758,12481.854295,3.448228e+06


In [18]:
changed_columns = df.columns.str.replace("'", '_')
changed_columns = changed_columns.str.replace('+', "")

In [19]:
df.columns = changed_columns
df

,External Reporting BU,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Q4_22,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.077714e+06,702338.98951,403290.97126,146782.43490,11452.84639,7186.77961,5490.00367,4452.80229,66650.186572,6825.696225,2.432185e+06
23,Digital Experience Total,3.593437e+05,203181.07510,92101.90304,32102.32943,8989.15503,12446.79748,2617.20363,2372.66414,4551.250486,2979.434210,7.206855e+05
36,Publishing Total,2.894870e+04,19649.15565,11566.49067,6730.47778,961.91266,295.11758,140.10942,69.10113,18.677990,0.000000,6.837974e+04
38,Digital Experience Total,3.497726e+04,21749.96103,12762.51346,4655.10681,1250.99980,660.27517,745.04432,576.33967,655.035250,79.645130,7.811218e+04
39,Digital Experience Total,4.999213e+04,40848.08746,31414.17946,23461.84333,40.06227,418.52159,36.48702,19.67807,38.461460,2597.078730,1.488665e+05
40,Grand Total inclusive of Magento/Marketo,1.550976e+06,987767.26875,551136.05789,213732.19225,22694.97615,21007.49143,9028.84806,7490.58530,71913.611758,12481.854295,3.448228e+06


In [20]:
# start by renaming the columns

In [21]:
df.columns

Index(['External Reporting BU', 'Q4_20', 'Q1_21', 'Q2_21', 'Q3_21', 'Q4_21',
       'Q1_22', 'Q2_22', 'Q3_22', 'Q4_22', 'As Performed / Upon Acceptance',
       'Total'],
      dtype='object')

In [22]:
df = df.groupby('External Reporting BU').sum()

In [23]:
df

,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Q4_22,As Performed / Upon Acceptance,Total
External Reporting BU,,,,,,,,,,,
Digital Experience Total,4.443131e+05,265779.12359,136278.59596,60219.27957,10280.21710,13525.59424,3398.73497,2968.68188,5244.747196,5656.158070,9.476642e+05
Digital Media Total,1.077714e+06,702338.98951,403290.97126,146782.43490,11452.84639,7186.77961,5490.00367,4452.80229,66650.186572,6825.696225,2.432185e+06
Grand Total inclusive of Magento/Marketo,1.550976e+06,987767.26875,551136.05789,213732.19225,22694.97615,21007.49143,9028.84806,7490.58530,71913.611758,12481.854295,3.448228e+06
Publishing Total,2.894870e+04,19649.15565,11566.49067,6730.47778,961.91266,295.11758,140.10942,69.10113,18.677990,0.000000,6.837974e+04


### Now that we have the data that is all numeric, we need to adjust for the reporting in thousands (FP&A report)

In [24]:
df = df * 1000

In [25]:
df

,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Q4_22,As Performed / Upon Acceptance,Total
External Reporting BU,,,,,,,,,,,
Digital Experience Total,4.443131e+08,2.657791e+08,1.362786e+08,6.021928e+07,10280217.10,13525594.24,3398734.97,2968681.88,5.244747e+06,5.656158e+06,9.476642e+08
Digital Media Total,1.077714e+09,7.023390e+08,4.032910e+08,1.467824e+08,11452846.39,7186779.61,5490003.67,4452802.29,6.665019e+07,6.825696e+06,2.432185e+09
Grand Total inclusive of Magento/Marketo,1.550976e+09,9.877673e+08,5.511361e+08,2.137322e+08,22694976.15,21007491.43,9028848.06,7490585.30,7.191361e+07,1.248185e+07,3.448228e+09
Publishing Total,2.894870e+07,1.964916e+07,1.156649e+07,6.730478e+06,961912.66,295117.58,140109.42,69101.13,1.867799e+04,0.000000e+00,6.837974e+07


### Creating the columns that have this amortization by period

# Below here is the first mistake - need to create a P00 column and then delete it! Such that the amortization does not include the actual period that is in there

In [26]:
new_columns = []
for i in range(12*3):
    if len(str(i))==1:
        new_column = 'P0'+ str(i)
    else:
        new_column = 'P'+ str(i)
    new_columns.append(new_column)

In [27]:
new_columns

['P00',
 'P01',
 'P02',
 'P03',
 'P04',
 'P05',
 'P06',
 'P07',
 'P08',
 'P09',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35']

In [28]:
qtrly_list = [col for col in df.columns if 'Q' in col]
qtrly_list

['Q4_20',
 'Q1_21',
 'Q2_21',
 'Q3_21',
 'Q4_21',
 'Q1_22',
 'Q2_22',
 'Q3_22',
 'Q4_22']

In [29]:
period_index = 0
for index, qtr in enumerate(qtrly_list):
    
    df[new_columns[period_index]]=df[qtr] / 3
    period_index +=1
    df[new_columns[period_index]]=df[qtr] / 3
    period_index +=1
    df[new_columns[period_index]]=df[qtr] / 3
    period_index +=1


In [30]:
df

,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Q4_22,As Performed / Upon Acceptance,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience Total,4.443131e+08,2.657791e+08,1.362786e+08,6.021928e+07,10280217.10,13525594.24,3398734.97,2968681.88,5.244747e+06,5.656158e+06,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media Total,1.077714e+09,7.023390e+08,4.032910e+08,1.467824e+08,11452846.39,7186779.61,5490003.67,4452802.29,6.665019e+07,6.825696e+06,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Grand Total inclusive of Magento/Marketo,1.550976e+09,9.877673e+08,5.511361e+08,2.137322e+08,22694976.15,21007491.43,9028848.06,7490585.30,7.191361e+07,1.248185e+07,...,7.002497e+06,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07
Publishing Total,2.894870e+07,1.964916e+07,1.156649e+07,6.730478e+06,961912.66,295117.58,140109.42,69101.13,1.867799e+04,0.000000e+00,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


## I Don't need the everything in this. I can now remove some of the details

First check that my periods match the quarterly deferred numbers/


In [31]:
df_qtrly_only = df.copy()
df_period_only = df.copy()

df_period_only = df_period_only.loc[:, df_period_only.columns.str.contains('P')]
df_qtrly_only = df_qtrly_only.loc[:, ~df_qtrly_only.columns.str.contains('P')]
                                  
df_period_only['total'] = df_period_only.sum(axis=1)
                            

In [32]:
df_period_only

,As Performed / Upon Acceptance,P00,P01,P02,P03,P04,P05,P06,P07,P08,...,P18,P19,P20,P21,P22,P23,P24,P25,P26,total
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience Total,5.656158e+06,1.481044e+08,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,...,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06,9.476642e+08
Digital Media Total,6.825696e+06,3.592379e+08,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,...,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07,2.432185e+09
Grand Total inclusive of Magento/Marketo,1.248185e+07,5.169918e+08,5.169918e+08,5.169918e+08,3.292558e+08,3.292558e+08,3.292558e+08,1.837120e+08,1.837120e+08,1.837120e+08,...,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07,3.448228e+09
Publishing Total,0.000000e+00,9.649567e+06,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,...,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03,6.837974e+07


In [33]:
df_qtrly_only

,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Q4_22,Total
External Reporting BU,,,,,,,,,,
Digital Experience Total,4.443131e+08,2.657791e+08,1.362786e+08,6.021928e+07,10280217.10,13525594.24,3398734.97,2968681.88,5.244747e+06,9.476642e+08
Digital Media Total,1.077714e+09,7.023390e+08,4.032910e+08,1.467824e+08,11452846.39,7186779.61,5490003.67,4452802.29,6.665019e+07,2.432185e+09
Grand Total inclusive of Magento/Marketo,1.550976e+09,9.877673e+08,5.511361e+08,2.137322e+08,22694976.15,21007491.43,9028848.06,7490585.30,7.191361e+07,3.448228e+09
Publishing Total,2.894870e+07,1.964916e+07,1.156649e+07,6.730478e+06,961912.66,295117.58,140109.42,69101.13,1.867799e+04,6.837974e+07


##### OK My periods work fine. Now I can move on to saving this and finishing the defered waterfall

In [34]:
df.columns

Index(['Q4_20', 'Q1_21', 'Q2_21', 'Q3_21', 'Q4_21', 'Q1_22', 'Q2_22', 'Q3_22',
       'Q4_22', 'As Performed / Upon Acceptance', 'Total', 'P00', 'P01', 'P02',
       'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12',
       'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22',
       'P23', 'P24', 'P25', 'P26'],
      dtype='object')

In [43]:
df = df.loc[:, df.columns.str.contains('P')]

In [44]:
df

,As Performed / Upon Acceptance,P00,P01,P02,P03,P04,P05,P06,P07,P08,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience Total,5.656158e+06,1.481044e+08,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media Total,6.825696e+06,3.592379e+08,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Grand Total inclusive of Magento/Marketo,1.248185e+07,5.169918e+08,5.169918e+08,5.169918e+08,3.292558e+08,3.292558e+08,3.292558e+08,1.837120e+08,1.837120e+08,1.837120e+08,...,7.002497e+06,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07
Publishing Total,0.000000e+00,9.649567e+06,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


### Now Drop P00 from the waterfall

In [45]:
df = df.drop('P00', axis=1)

In [46]:
df

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience Total,5.656158e+06,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,2.007309e+07,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media Total,6.825696e+06,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,4.892748e+07,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Grand Total inclusive of Magento/Marketo,1.248185e+07,5.169918e+08,5.169918e+08,3.292558e+08,3.292558e+08,3.292558e+08,1.837120e+08,1.837120e+08,1.837120e+08,7.124406e+07,...,7.002497e+06,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07
Publishing Total,0.000000e+00,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,2.243493e+06,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


In [47]:
config_dict = {
    "ADBE_cal": {
        "direct_filename": "../Data/OLD/ADOBE_FINANCIAL_CALENDAR.xlsx",
        "sheetname": "ADBE_cal"
    },
    "FX_forwards": {
        "filename": "FX_forward_rates.xlsx",
        "sheetname": "forward_data"
    },
    "FX_rates": {
        "filename": "FX_data.xlsx",
        "sheetname": "to_matlab"
    },
    "billings": {
        "filename": "all_billings_inputs.xlsx",
        "base_sheetname": "base_billings",
        "type_A_sheetname": "type_A_no_config"
    },
    "bookings": {
        "filename": "2020_bookings_fcst_Q2.xlsx",
        "sheetname": "source"
    },
    "curr_map": {
        "filename": "currency_map.xlsx",
        "sheetname": "curr_map"
    },
    "list_future_periods": [
        "2020-10",
        "2020-11",
        "2020-12",
        "2021-01",
        "2021-02",
        "2021-03",
        "2021-04",
        "2021-05",
        "2021-06",
        "2021-07",
        "2021-08",
        "2021-09"
    ],
    "path_to_data": "../data/Data_2020_P09/",
    "output_dir": {
        "intermediate": "../data/Data_2020_P09/processed/intermediate/",
        "final": "../data/Data_2020_P09/processed/final/"
    },
    "deferred_workbook": {
        "filename": "Q3'20 Rev Acctg Mgmt Workbook (09-03-20).xlsx",
        "sheetname": "Deferred Revenue Forecast",
        "skiprows": 5,
        "keeper_rows": [
            "Digital Media Total",
              "Publishing Total",
              "Digital Experience Total",
              "Marketo Deferred",
              "Magento Deferred",
              "Grand Total inclusive of Magento/Marketo"
        ],
        "str_replace_from": ["Magento Deferred", "Marketo Deferred"],
        "str_replace_to": "Digital Experience Total",
        "drop_columns": [
            "Major Product Config", " Historical"
        ]
    },
    "bookings_BU_mapping": {"Creative": "Digital Media",
                            "Document Cloud": "Digital Media",
                            "DX Other": "Digital Experience",
                            "Experience Cloud": "Digital Experience",
                            "Print & Publishing" : "Publishing"
    },
    "waterfall_BU_mapping": {"Digital Experience Total": "Digital Experience",
                            "Digital Media Total": "Digital Media",
                            "Publishing Total": "Publishing"         
    },
    "waterfall_drop": "Grand Total inclusive of Magento/Marketo"
}

In [40]:
config_dict

{'ADBE_cal': {'direct_filename': '../Data/OLD/ADOBE_FINANCIAL_CALENDAR.xlsx',
  'sheetname': 'ADBE_cal'},
 'FX_forwards': {'filename': 'FX_forward_rates.xlsx',
  'sheetname': 'forward_data'},
 'FX_rates': {'filename': 'FX_data.xlsx', 'sheetname': 'to_matlab'},
 'billings': {'filename': 'all_billings_inputs.xlsx',
  'base_sheetname': 'base_billings',
  'type_A_sheetname': 'type_A_no_config'},
 'bookings': {'filename': '2020_bookings_fcst_Q2.xlsx', 'sheetname': 'source'},
 'curr_map': {'filename': 'currency_map.xlsx', 'sheetname': 'curr_map'},
 'list_future_periods': ['2020-10',
  '2020-11',
  '2020-12',
  '2021-01',
  '2021-02',
  '2021-03',
  '2021-04',
  '2021-05',
  '2021-06',
  '2021-07',
  '2021-08',
  '2021-09'],
 'path_to_data': '../data/Data_2020_P09/',
 'output_dir': {'intermediate': '../data/Data_2020_P09/processed/intermediate/',
  'final': '../data/Data_2020_P09/processed/final/'},
 'deferred_workbook': {'filename': "Q3'20 Rev Acctg Mgmt Workbook (09-03-20).xlsx",
  'sheetna

In [48]:
df.index

Index(['Digital Experience Total', 'Digital Media Total',
       'Grand Total inclusive of Magento/Marketo', 'Publishing Total'],
      dtype='object', name='External Reporting BU')

In [49]:
df

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience Total,5.656158e+06,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,2.007309e+07,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media Total,6.825696e+06,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,4.892748e+07,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Grand Total inclusive of Magento/Marketo,1.248185e+07,5.169918e+08,5.169918e+08,3.292558e+08,3.292558e+08,3.292558e+08,1.837120e+08,1.837120e+08,1.837120e+08,7.124406e+07,...,7.002497e+06,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07
Publishing Total,0.000000e+00,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,2.243493e+06,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


In [50]:
df = df.rename(index=config_dict['waterfall_BU_mapping'])

In [51]:
df

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,5.656158e+06,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,2.007309e+07,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media,6.825696e+06,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,4.892748e+07,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Grand Total inclusive of Magento/Marketo,1.248185e+07,5.169918e+08,5.169918e+08,3.292558e+08,3.292558e+08,3.292558e+08,1.837120e+08,1.837120e+08,1.837120e+08,7.124406e+07,...,7.002497e+06,3.009616e+06,3.009616e+06,3.009616e+06,2.496862e+06,2.496862e+06,2.496862e+06,2.397120e+07,2.397120e+07,2.397120e+07
Publishing,0.000000e+00,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,2.243493e+06,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


In [52]:
df = df.drop(config_dict['waterfall_drop'])

In [53]:
df

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,5.656158e+06,1.481044e+08,1.481044e+08,8.859304e+07,8.859304e+07,8.859304e+07,4.542620e+07,4.542620e+07,4.542620e+07,2.007309e+07,...,4.508531e+06,1.132912e+06,1.132912e+06,1.132912e+06,9.895606e+05,9.895606e+05,9.895606e+05,1.748249e+06,1.748249e+06,1.748249e+06
Digital Media,6.825696e+06,3.592379e+08,3.592379e+08,2.341130e+08,2.341130e+08,2.341130e+08,1.344303e+08,1.344303e+08,1.344303e+08,4.892748e+07,...,2.395593e+06,1.830001e+06,1.830001e+06,1.830001e+06,1.484267e+06,1.484267e+06,1.484267e+06,2.221673e+07,2.221673e+07,2.221673e+07
Publishing,0.000000e+00,9.649567e+06,9.649567e+06,6.549719e+06,6.549719e+06,6.549719e+06,3.855497e+06,3.855497e+06,3.855497e+06,2.243493e+06,...,9.837253e+04,4.670314e+04,4.670314e+04,4.670314e+04,2.303371e+04,2.303371e+04,2.303371e+04,6.225997e+03,6.225997e+03,6.225997e+03


### Saving the waterfall as Q2_waterfall

In [ ]:
pickle.dump(df_waterfall, open('../data/processed/Q2_waterfall.p', 'wb'))

### This will get loaded into the deferred revenue forecast

In [ ]:
bookings_BU_mapping={"Creative": "Digital Media",
                            "Document Cloud": "Digital Media",
                            "DX Other": "Digital Experience",
                            "Experience Cloud": "Digital Experience",
                            "Print & Publishing": "Publishing"
                    }

In [ ]:
bookings_BU_mapping


In [ ]:
for key, value in bookings_BU_mapping.items():
    print(key, value)
    

# Below is old work on the trend. We don't need this now

In [ ]:
'''

BELOW IS NOT NEEDED, BUT MAYBE IN THE FUTURE??

def clean_DR_history(df):
    
    df = df.rename(columns={'External Reporting BU': 'BU',
                       'Major Product Config': 'MPC',
                       'Q/Q change': 'q_over_q',
                       'Y/Y change': 'y_over_y'})
    df = df.drop(columns = ['Unnamed: 13', 'Unnamed: 14'])
    
    df['BU'] = df['BU'].fillna(method='ffill')
    
    df['BU'] = df['BU'].str.strip()
    df['MPC'] = df['MPC'].str.strip()

    df = df.dropna(subset=['MPC'], axis=0)
    df = df.drop_duplicates(subset=['BU','MPC'], keep='first')
    df = df.set_index(['BU', 'MPC'])
    
    old_columns = df.columns
    new_columns = []
    for col in old_columns:
        new_columns.append(col.replace("'", '_'))
        
    columns_dict = dict(zip(old_columns, new_columns))
    df.rename(columns = columns_dict, inplace=True)
    
    return df
'''

In [ ]:
#df = pd.read_excel("../data/Data_2020_P06/Q2'20 Rev Acctg Mgmt Workbook (06-04-20).xlsx", 
#                   sheet_name='Deferred Revenue Trend', skiprows=5)

In [ ]:
#df_hist = clean_DR_history(df)

In [ ]:
#df_hist.head(40)